In [1]:
import pandas as pd
from pyserini.search.lucene import LuceneSearcher
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords

d:\Hafiz\Kuliah\Semester 5\Information Retrieval\Projek\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_colwidth', 500)

In [3]:
istilah_kesehatan = {
    # Tekanan Darah
    "hipertensi": "tekanan darah tinggi",
    "hipotensi": "tekanan darah rendah",

    # Gula Darah
    "hiperglikemia": "kadar gula darah tinggi",
    "hipoglikemia": "kadar gula darah rendah",
    "diabetes melitus": "penyakit kencing manis",

    # Pernapasan
    "dispnea": "sesak napas",
    "rinorea": "pilek atau ingusan",
    "epistaksis": "mimisan",
    "faringitis": "radang tenggorokan",
    "laringitis": "radang pita suara",
    "asma": "bengek",

    # Pencernaan
    "dispepsia": "mag atau gangguan pencernaan",
    "konstipasi": "sembelit atau susah buang air besar",
    "diare": "mencret",
    "hemoroid": "wasir atau ambeien",
    "apendisitis": "radang usus buntu",
    "gastritis": "radang lambung",

    # Kulit dan Alergi
    "urtikaria": "biduran atau kaligata",
    "dermatitis": "eksim atau radang kulit",
    "varisela": "cacar air",
    "morbili": "campak atau tampek",
    "veruka": "kutil",
    "tinea pedis": "kutu air",
    "lotion": "losion",

    # Kepala dan Saraf
    "sefalgia": "sakit kepala atau pusing",
    "insomnia": "susah tidur",
    "sinkop": "pingsan",
    "konvulsi": "kejang",

    # Umum
    "pireksia": "demam",
    "hipertermia": "suhu tubuh sangat tinggi",
    "hipotermia": "suhu tubuh sangat rendah",
    "mialgia": "nyeri otot",
    "artralgia": "nyeri sendi",
    "fatik": "kelelahan atau rasa capai",
    "edema": "bengkak",
    "pruritus": "gatal-gatal",
    "anemia": "kurang darah",
    "karsinoma": "kanker",
    "neoplasma": "tumor",
    "halitosis": "bau mulut",
    "kalkulus renal": "batu ginjal",
    "moisturizing": "moisturizer"
}

In [4]:
stop_words = set(stopwords.words('indonesian'))

In [5]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk stemming
def stem_tokens(tokens):
    lemmas = [stemmer.stem(token) for token in tokens]
    return lemmas

In [6]:
def normalize_text(text, dictionary):
    tokens = text.split()

    normalized_tokens = []
    for token in tokens:
        normalized_token = dictionary.get(token, token)
        normalized_tokens.append(normalized_token)

    return " ".join(normalized_tokens)

In [7]:
def preprocess_query(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = normalize_text(text, istilah_kesehatan)

    return text

In [8]:
queries = [
  'gejala diabetes',
  'makanan sehat untuk jantung',
  'kesehatan mental',
  'efek samping vaksin covid-19',
  'manfaat buah'
]

In [9]:
queries = [preprocess_query(query) for query in queries]

In [10]:
print(queries)

['gejala diabetes', 'makanan sehat jantung', 'kesehatan mental', 'efek samping vaksin covid-19', 'manfaat buah']


# Membaca Dataframe

In [11]:
df = pd.read_csv('../artikel/raw_health_articles.csv')
df.drop(columns=['description', 'content'], inplace=True)
df.head()

,article_index,title,tag,link,date,source
0,1,5 Cara Merawat Ginjal agar Sehat & Cegah Penyakit Ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-cara-merawat-ginjal-agar-sehat-cegah-penyakit-ginjal,3 September 2025,Biofarma
1,2,7 Tips Untuk Menjaga Kesehatan Mata,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/7-tips-untuk-menjaga-kesehatan-mata,29 Agustus 2025,Biofarma
2,3,Jangan Sepelekan! 10 Penyebab Campak yang Sering Terabaikan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/jangan-sepelekan-10-penyebab-campak-yang-sering-terabaikan,28 Agustus 2025,Biofarma
3,4,"Campak - Penyebab, Gejala, dan Pencegahannya",Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,25 Agustus 2025,Biofarma
4,5,Cara Mudah Menjaga Kesehatan Gigi dan Mulut agar Terhindar dari Penyakit,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/cara-mudah-menjaga-kesehatan-gigi-dan-mulut-agar-terhindar-dari-penyakit,20 Agustus 2025,Biofarma


In [12]:
# Fungsi untuk print hasil
def display_results(query, hits):
    print(f'\n{query}')

    if not hits:
        print("Tidak ada dokumen yang sesuai.")
        return
    
    results_with_scores = []
    
    for i in range(len(hits)):
        matching_row = df[df['article_index'] == int(hits[i].docid)].copy()
        
        if not matching_row.empty:
            # Add score to the dataframe row
            matching_row = matching_row.copy()
            matching_row['score'] = hits[i].score
            matching_row['rank'] = i + 1
            results_with_scores.append(matching_row)
            
            # print(f'{i+1:2} ID: {hits[i].docid:4} Score: {hits[i].score:.5f}')
            # print(f'   Title: {matching_row.iloc[0]["title"]}')
            # print()
    
    if results_with_scores:
        result_df = pd.concat(results_with_scores, ignore_index=True)
        return result_df
    else:
        print("No matching documents found in DataFrame.")
        return pd.DataFrame()

## Bm25 Retrieval (Title)

In [13]:
searcher_title = LuceneSearcher('../indexing/indexes/title_jsonl')
searcher_content_title = LuceneSearcher('../indexing/indexes/title_content_jsonl')
searcher_desc_content = LuceneSearcher('../indexing/indexes/title_desc_content_jsonl')

In [21]:
# Test the function with a query
query = queries[2]  
hits = searcher_title.search(query, k=100)  

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

print("\nResult DataFrame with scores:")
result_df


kesehatan mental

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,251,Ini Cara Menerapkan Mindfulness untuk Jaga Kesehatan Mental,"Kesehatan Mental, Depresi",https://www.halodoc.com/artikel/ini-cara-menerapkan-mindfulness-untuk-jaga-kesehatan-mental,12 September 2025,halodoc,2.490600,1
1,59,Manfaat Puasa Bagi Kesehatan: Secara Fisik dan Mental,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/manfaat-puasa-bagi-kesehatan-secara-fisik-dan-mental,12 Maret 2025,Biofarma,2.490599,2
2,208,"Cerita Reza Chandika: Pandemi, Transformasi Mental dan Thanos",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/cerita-reza-chandika-pandemi-transformasi-mental-dan-thanos,14 Oktober 2022,Klikdokter,2.345200,3
3,71,"Menjaga Kesehatan Mental Remaja: Tantangan, Faktor Penyebab, dan Solusi",Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/menjaga-kesehatan-mental-remaja-tantangan-faktor-penyebab-dan-solusi,3 Februari 2025,Biofarma,2.278800,4


## Bm25 Retrieval (Title + Content)

In [27]:
# Test the function with a query
query = queries[2]  
hits = searcher_content_title.search(query, k=10)  

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

print("\nResult DataFrame with scores:")
result_df


kesehatan mental

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,71,"Menjaga Kesehatan Mental Remaja: Tantangan, Faktor Penyebab, dan Solusi",Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/menjaga-kesehatan-mental-remaja-tantangan-faktor-penyebab-dan-solusi,3 Februari 2025,Biofarma,1.9302,1
1,302,7 Manfaat Buah Pinang Muda untuk Pria yang Perlu Diketahui,KESEHATAN,https://www.alodokter.com/7-manfaat-buah-pinang-muda-untuk-pria-yang-perlu-diketahui,16 September 2025,alodokter,1.8241,2
2,251,Ini Cara Menerapkan Mindfulness untuk Jaga Kesehatan Mental,"Kesehatan Mental, Depresi",https://www.halodoc.com/artikel/ini-cara-menerapkan-mindfulness-untuk-jaga-kesehatan-mental,12 September 2025,halodoc,1.8034,3
3,208,"Cerita Reza Chandika: Pandemi, Transformasi Mental dan Thanos",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/cerita-reza-chandika-pandemi-transformasi-mental-dan-thanos,14 Oktober 2022,Klikdokter,1.7917,4
4,213,Jenis-Jenis Depresi yang Perlu Kamu Tahu,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/jenis-jenis-depresi-yang-perlu-anda-tahu,Mei Oktober 2022,Klikdokter,1.7283,5
5,65,Ingin Tubuh Sehat dan Pikiran Tenang? Ini Manfaat Kebugaran Jasmani!,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/ingin-tubuh-sehat-dan-pikiran-tenang-ini-manfaat-kebugaran-jasmani,19 Februari 2025,Biofarma,1.7176,6
6,59,Manfaat Puasa Bagi Kesehatan: Secara Fisik dan Mental,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/manfaat-puasa-bagi-kesehatan-secara-fisik-dan-mental,12 Maret 2025,Biofarma,1.7131,7
7,151,"Hai Gen Z, Punya Asuransi Kesehatan Penting Untuk Masa Depan Yang Lebih Aman",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/gen-z-penting-punya-asuransi-kesehatan,Juli Desember 2023,Klikdokter,1.6903,8
8,324,10 Shampoo untuk Kulit Kepala Gatal yang Ampuh,HIDUP SEHAT,https://www.alodokter.com/shampoo-untuk-kulit-kepala-gatal-yang-ampuh,11 September 2025,alodokter,1.6576,9
9,244,Doxing: Kenali Bahaya dan Cara Lindungi Diri,Kesehatan Mental,https://www.halodoc.com/artikel/doxing-kenali-bahaya-dan-cara-lindungi-diri,12 September 2025,halodoc,1.6553,10


## Bm25 Retrieval (Title + Desc + Content)

In [18]:
# Test the function with a query
query = queries[0] 
hits = searcher_desc_content.search(query, k=100)  

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

print("\nResult DataFrame with scores:")
result_df


gejala diabetes

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,86,Gejala Diabetes Melitus yang Harus Kamu Ketahui!,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/gejala-diabetes-melitus-yang-harus-kamu-ketahui,11 Desember 2024,Biofarma,2.2057,1
1,49,5 Faktor Risiko Diabetes yang Perlu Anda Ketahui,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-faktor-risiko-diabetes-yang-perlu-anda-ketahui,25 Maret 2025,Biofarma,2.1629,2
2,410,"Kisah Tragis Sekeluarga Terpapar Radiasi Tinggi, Cuma Ayah yang Selamat",Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8112921/kisah-tragis-sekeluarga-terpapar-radiasi-tinggi-cuma-ayah-yang-selamat,15 September 2025,detik,2.0167,3
3,353,7 Menu Diet Sehat Seminggu yang Bernutrisi,HIDUP SEHAT,https://www.alodokter.com/7-menu-diet-sehat-seminggu-yang-bernutrisi,10 September 2025,alodokter,1.9762,4
4,201,"Hari Diabetes Sedunia, Diabetasol Gelar Edukasi Kesehatan",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/hari-diabetes-sedunia-diabetasol,Oktober November 2022,Klikdokter,1.8422,5
...,...,...,...,...,...,...,...,...
95,220,Sakit Perut Sebelah Kanan Bawah Sampai ke Pinggang: Penyebab dan Cara Mengatasinya,Sakit Pinggang,https://www.halodoc.com/artikel/sakit-perut-sebelah-kanan-bawah-sampai-ke-pinggang-penyebab-dan-cara-mengatasinya,15 September 2025,halodoc,0.8515,96
96,104,8 Gejala Penyakit Influenza yang Harus Diwaspadai!,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/8-gejala-penyakit-influenza-yang-harus-diwaspadai,17 Oktober 2024,Biofarma,0.8492,97
97,284,Cara Mengeluarkan Dahak pada Bayi yang Bisa Dicoba di Rumah,Batuk,https://www.halodoc.com/artikel/cara-mengeluarkan-dahak-pada-bayi-yang-bisa-dicoba-di-rumah,11 September 2025,halodoc,0.8335,98
98,279,Rokok Kretek: Kandungan dan Bahayanya untuk Kesehatan,Hidup Sehat,https://www.halodoc.com/artikel/rokok-kretek-kandungan-dan-bahayanya-untuk-kesehatan,11 September 2025,halodoc,0.8328,99
